# AMS-DB: Building Agents with Ollama + Graphiti Integration

This notebook demonstrates how to use AMS-DB's integrated Ollama + Graphiti framework to build intelligent agents with persistent memory and local LLM capabilities.

## Key Features Covered:
- 🤖 **Agent Configuration** - Creating agents with AMS-DB's config system
- 🧠 **Local LLMs** - Running Ollama models for privacy and cost savings  
- 📊 **High-Speed Database** - Polars-based conversation and knowledge storage
- 🕸️ **Knowledge Graphs** - Graphiti temporal knowledge graphs for memory
- 💬 **Multi-Agent Conversations** - Generate training data and agent interactions

## Prerequisites

Before starting, ensure you have:
1. **Ollama installed and running**: `ollama serve`
2. **Neo4j running**: Docker or local installation
3. **AMS-DB environment**: Virtual environment with dependencies

```bash
# Install Ollama models
ollama pull phi4:latest          # Main LLM
ollama pull gemma3:4b           # Small/fast model  
ollama pull nomic-embed-text    # Embedding model

# Start Neo4j (Docker example)
docker run -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password neo4j:latest
```

In [ ]:
# Import AMS-DB components
import sys
import os
sys.path.append(os.path.abspath(os.path.join('.', '..', '..')))

from src.ams_db.core.polars_db import PolarsDBHandler
from src.ams_db.core.base_agent_config import AgentConfig
from src.ams_db.core.graphiti_pipe import GraphitiRAGFramework
from src.ams_db.core.conversation_generator import ConversationGenerator

import asyncio
import json
import logging
from datetime import datetime, timezone
from pathlib import Path
import uuid

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ AMS-DB components imported successfully!")
print("📦 Available components:")
print("   • PolarsDBHandler - High-speed database operations")
print("   • AgentConfig - Agent configuration management")
print("   • GraphitiRAGFramework - Knowledge graph + Ollama integration")
print("   • ConversationGenerator - Multi-agent conversation system")

## 🚀 Part 1: Initialize AMS-DB with Ollama + Graphiti

AMS-DB provides a unified framework that combines:
- **Polars Database** for high-speed data operations
- **Ollama** for local LLM processing (privacy + cost savings)
- **Graphiti** for temporal knowledge graphs and memory
- **Agent Config System** for complete agent management

This integration allows you to build sophisticated agents with persistent memory using local models.

In [ ]:
# Initialize AMS-DB components with Ollama configuration
print("🔧 Initializing AMS-DB with Ollama + Graphiti...")

# 1. Initialize Polars database for high-speed operations
db_handler = PolarsDBHandler("ams_agent_database")
print("✅ Polars database initialized")

# 2. Initialize Graphiti RAG Framework with Ollama
graphiti_framework = GraphitiRAGFramework(
    neo4j_uri="bolt://localhost:7687",
    neo4j_user="neo4j", 
    neo4j_password="password",
    db_path="ams_agent_database",
    ollama_base_url="http://localhost:11434/v1",
    llm_model="phi4:latest",           # Main reasoning model
    small_model="gemma3:4b",           # Fast response model
    embedding_model="nomic-embed-text", # Local embeddings
    embedding_dim=768
)
print("✅ Graphiti RAG Framework with Ollama initialized")

# 3. Initialize conversation generator for multi-agent interactions
conversation_generator = ConversationGenerator(db_handler, graphiti_framework)
print("✅ Multi-agent conversation generator ready")

print("\n🎉 AMS-DB system fully initialized!")
print("   • Database: Polars high-speed backend")
print("   • LLM: Ollama local models")
print("   • Memory: Graphiti knowledge graphs")
print("   • Agent System: Complete config management")

## 🤖 Part 2: Create Agents with AMS-DB Configuration System

AMS-DB's agent configuration system provides complete control over:
- **Prompts** - System prompts, boosters, prime directives
- **Modalities** - Vision, speech, LaTeX, embeddings  
- **Models** - LLM configs, embedding settings
- **Databases** - Knowledge bases, conversation history
- **Export/Import** - Save and share agent configurations

Let's create a specialized "Research Assistant" agent that uses Ollama + Graphiti for persistent memory.

In [ ]:
# Create a Research Assistant agent with AMS-DB configuration
print("🔬 Creating Research Assistant Agent...")

# 1. Initialize agent configuration
research_agent = AgentConfig("research_assistant_001")

# 2. Set comprehensive system prompt
system_prompt = """
You are an advanced Research Assistant powered by local Ollama models and Graphiti knowledge graphs.

Your capabilities include:
🔍 Deep research and analysis across multiple domains
🧠 Persistent memory through knowledge graph integration  
📊 Data synthesis and pattern recognition
🤝 Collaborative research with other agents
📚 Continuous learning from every interaction

You maintain context across conversations and build knowledge incrementally.
Always cite your sources and explain your reasoning process.
"""

research_agent.set_prompt("llmSystem", system_prompt.strip())
research_agent.set_prompt("primeDirective", 
    "Conduct thorough research, maintain accurate knowledge graphs, "
    "and provide evidence-based insights with full transparency.")

# 3. Configure modalities for multimodal capabilities
research_agent.set_modality_flag("LLM_SYSTEM_PROMPT_FLAG", True)
research_agent.set_modality_flag("AGENT_FLAG", True)  
research_agent.set_modality_flag("EMBEDDING_FLAG", True)  # Enable knowledge search
research_agent.set_modality_flag("LATEX_FLAG", True)     # Scientific notation
research_agent.set_modality_flag("LLAVA_FLAG", True)     # Vision for charts/diagrams

# 4. Configure for Ollama models
research_agent.update_config({
    "model_config": {
        "largeLanguageModel": {
            "names": ["phi4:latest"],
            "instances": ["ollama_primary"],
            "model_config_template": {
                "base_url": "http://localhost:11434/v1",
                "api_key": None,
                "temperature": 0.7,
                "max_tokens": 4096
            }
        },
        "embedding": {
            "names": ["nomic-embed-text"],
            "instances": ["ollama_embedding"],
            "model_config_template": {
                "base_url": "http://localhost:11434/v1",
                "embedding_dim": 768
            }
        }
    }
})

# 5. Add agent to database with Graphiti integration
agent_id = graphiti_framework.create_agent(
    agent_config=research_agent.get_config(),
    agent_name="Research Assistant",
    description="Advanced research agent with Ollama + Graphiti integration",
    tags=["research", "ollama", "graphiti", "multimodal"]
)

print(f"✅ Research Assistant created: {agent_id}")
print(f"🧠 System prompt: {len(system_prompt)} characters")
print(f"🎛️ Modalities enabled: {sum(research_agent.get_config()['agent_core']['modalityFlags'].values())}")
print(f"🤖 Model: Ollama phi4:latest with local embeddings")

# 6. Load agent into Graphiti framework
success = graphiti_framework.load_agent(agent_id)
print(f"🕸️ Graphiti integration: {'✅ Active' if success else '❌ Failed'}")

## 📚 Part 3: Add Knowledge to Agent with Graphiti Integration

Now we'll add knowledge to our Research Assistant that will be:
1. **Stored in Polars** for high-speed querying
2. **Embedded with Ollama** using local nomic-embed-text model
3. **Indexed in Graphiti** for temporal knowledge graph capabilities
4. **Available for RAG** during conversations and research

In [ ]:
# Add research knowledge to the agent with Graphiti + Ollama integration
print("📖 Adding knowledge base to Research Assistant...")

# Sample research knowledge entries
research_knowledge = [
    {
        "title": "Transformer Architecture Overview",
        "content": """
        The Transformer architecture, introduced in 'Attention Is All You Need' (Vaswani et al., 2017),
        revolutionized natural language processing through the self-attention mechanism.
        
        Key components:
        - Self-attention layers enable parallel processing
        - Positional encoding provides sequence information
        - Multi-head attention captures different representation subspaces
        - Feed-forward networks process attended information
        
        Applications: GPT, BERT, T5, and most modern LLMs are based on Transformer variants.
        """,
        "source": "AI Research",
        "tags": ["transformer", "attention", "nlp", "architecture"]
    },
    {
        "title": "Knowledge Graph Applications in AI",
        "content": """
        Knowledge graphs represent structured relationships between entities, enabling:
        
        1. Semantic Understanding: Capture real-world relationships
        2. Reasoning: Infer new facts from existing knowledge  
        3. Explainability: Provide transparent decision paths
        4. Context Preservation: Maintain temporal and causal relationships
        
        Modern applications:
        - RAG systems for enhanced LLM responses
        - Recommendation engines with entity relationships
        - Drug discovery through biochemical pathway modeling
        - Financial risk assessment via entity connections
        """,
        "source": "Graph Research",
        "tags": ["knowledge_graph", "reasoning", "rag", "relationships"]
    },
    {
        "title": "Local LLM Benefits with Ollama",
        "content": """
        Running local LLMs with Ollama provides significant advantages:
        
        Privacy Benefits:
        - Complete data control and sovereignty
        - No external API calls or data transmission
        - GDPR and compliance-friendly deployment
        
        Cost Benefits:  
        - No per-token or API usage fees
        - Predictable infrastructure costs
        - Scalable without variable pricing
        
        Performance Benefits:
        - Reduced latency for local processing
        - No network dependency or rate limits
        - Customizable model parameters and fine-tuning
        """,
        "source": "Local AI Research", 
        "tags": ["ollama", "privacy", "local_llm", "cost_efficiency"]
    }
]

# Add knowledge with Ollama embeddings to both Polars and Graphiti
async def add_knowledge_with_integration():
    for i, knowledge in enumerate(research_knowledge, 1):
        print(f"📚 Adding knowledge {i}/{len(research_knowledge)}: {knowledge['title']}")
        
        # 1. Add to Polars database for fast querying
        kb_id = db_handler.add_knowledge_document(
            agent_id=agent_id,
            title=knowledge["title"],
            content=knowledge["content"],
            content_type="text",
            source=knowledge["source"],
            tags=knowledge["tags"],
            metadata={
                "embedded_with": "ollama_nomic_embed_text",
                "integration": "graphiti_rag",
                "knowledge_type": "research"
            }
        )
        
        # 2. Add to Graphiti with Ollama embeddings
        await graphiti_framework.add_knowledge_with_embedding(
            title=knowledge["title"],
            content=knowledge["content"],
            source=knowledge["source"], 
            tags=knowledge["tags"]
        )
        
        print(f"   ✅ Added to Polars (ID: {kb_id}) and Graphiti knowledge graph")

# Run the async knowledge integration
await add_knowledge_with_integration()

print(f"\n🎉 Knowledge integration complete!")
print(f"   • Knowledge entries: {len(research_knowledge)}")
print(f"   • Embeddings: Ollama nomic-embed-text (local)")
print(f"   • Storage: Polars database + Graphiti graph")
print(f"   • Agent: {agent_id} ready for research conversations")

# Show knowledge statistics
knowledge_stats = db_handler.search_knowledge_base(agent_id, "transformer")
print(f"   • Test search 'transformer': {knowledge_stats.height} results found")

## 💬 Part 4: Conversation with Memory and Multi-Agent Interactions

Now let's demonstrate the powerful conversation capabilities:
1. **Single-agent conversations** with persistent memory via Graphiti
2. **Multi-agent interactions** between different specialized agents
3. **Knowledge-enhanced responses** using RAG with local Ollama embeddings
4. **Training data generation** for fine-tuning and dataset creation

All powered by local Ollama models for privacy and Graphiti for temporal memory.

In [ ]:
# Demonstrate conversation with memory and knowledge integration
print("🗣️ Starting conversation with Research Assistant...")

async def demonstrate_conversation_with_memory():
    """Demonstrate conversation with persistent memory and knowledge integration."""
    
    session_id = f"research_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    print(f"📝 Session ID: {session_id}")
    
    # Conversation topics that will trigger knowledge retrieval
    conversation_turns = [
        {
            "user": "Can you explain the key innovations in Transformer architecture?",
            "context": "This should retrieve and use our knowledge about Transformers"
        },
        {
            "user": "How do knowledge graphs enhance AI systems?", 
            "context": "This should access our knowledge graph research"
        },
        {
            "user": "What are the benefits of using local LLMs like Ollama?",
            "context": "This should reference our Ollama benefits knowledge"
        },
        {
            "user": "How do these three concepts work together in a research system?",
            "context": "This should synthesize across all previous knowledge and conversation"
        }
    ]
    
    for i, turn in enumerate(conversation_turns, 1):
        print(f"\n--- Turn {i}/{len(conversation_turns)} ---")
        print(f"👤 User: {turn['user']}")
        print(f"💭 Expected: {turn['context']}")
        
        # 1. Store user message in conversation history
        user_msg_id = db_handler.add_conversation_message(
            agent_id=agent_id,
            role="user", 
            content=turn["user"],
            session_id=session_id,
            metadata={"turn": i, "type": "research_query"}
        )
        
        # 2. Search knowledge base with Ollama embeddings
        relevant_knowledge = await graphiti_framework.search_knowledge_with_context(
            query=turn["user"],
            limit=3,
            include_graph_context=True
        )
        
        print(f"🔍 Found {len(relevant_knowledge)} relevant knowledge entries")
        for j, knowledge in enumerate(relevant_knowledge[:2], 1):
            title = knowledge.get('title', f'Knowledge {j}')
            print(f"   {j}. {title}")
        
        # 3. Add conversation turn to Graphiti (builds temporal memory)
        await graphiti_framework.add_conversation_turn(
            user_input=turn["user"],
            assistant_response="[Research Assistant response with knowledge integration]",
            session_id=session_id,
            metadata={"knowledge_entries": len(relevant_knowledge)}
        )
        
        # 4. Store assistant response in database
        assistant_response = f"""Based on my knowledge base and previous research, I can provide insights on {turn['user'].lower()}.

[In a real implementation, this would be generated by Ollama phi4:latest using the retrieved knowledge and conversation context from Graphiti]

Key points from knowledge base:
{chr(10).join([f"• {k.get('title', 'Knowledge')}" for k in relevant_knowledge[:3]])}

This response maintains context from our previous {i-1} conversation turns and builds upon our growing knowledge graph."""
        
        assistant_msg_id = db_handler.add_conversation_message(
            agent_id=agent_id,
            role="assistant",
            content=assistant_response,
            session_id=session_id,
            metadata={"turn": i, "knowledge_used": len(relevant_knowledge)}
        )
        
        print(f"🤖 Research Assistant: {assistant_response[:100]}...")
        print(f"💾 Stored in conversation history (User: {user_msg_id}, Assistant: {assistant_msg_id})")

# Run the conversation demonstration
await demonstrate_conversation_with_memory()

print(f"\n🎉 Conversation demonstration complete!")
print("✅ Features demonstrated:")
print("   • Knowledge retrieval with Ollama embeddings") 
print("   • Temporal memory through Graphiti integration")
print("   • High-speed conversation storage in Polars")
print("   • Context accumulation across conversation turns")
print("   • RAG-enhanced responses with local models")

## 🤝 Part 5: Multi-Agent Conversation Generation

AMS-DB's conversation generator can create multi-agent interactions for:
- **Training Data**: Generate diverse conversation datasets for model fine-tuning
- **Agent Testing**: Validate agent personalities and knowledge integration
- **Research Simulation**: Model collaborative research scenarios
- **Data Export**: Create JSONL datasets for machine learning workflows

Let's create additional agents and generate multi-agent conversations.

In [ ]:
# Create additional agents and generate multi-agent conversations
print("👥 Creating additional agents for multi-agent conversations...")

# Create a Technical Expert agent
tech_expert = AgentConfig("tech_expert_001")
tech_expert.set_prompt("llmSystem", 
    "You are a Technical Expert specializing in AI/ML implementation details. "
    "You focus on practical aspects, code examples, and technical best practices. "
    "You work well with researchers to bridge theory and implementation.")
tech_expert.set_modality_flag("LLM_SYSTEM_PROMPT_FLAG", True)
tech_expert.set_modality_flag("AGENT_FLAG", True)

tech_expert_id = graphiti_framework.create_agent(
    agent_config=tech_expert.get_config(),
    agent_name="Technical Expert",
    description="Implementation-focused AI/ML expert",
    tags=["technical", "implementation", "coding"]
)

# Create a Creative Strategist agent  
creative_strategist = AgentConfig("creative_strategist_001")
creative_strategist.set_prompt("llmSystem",
    "You are a Creative Strategist who thinks outside the box about AI applications. "
    "You focus on innovative use cases, user experience, and strategic implications. "
    "You bring creative perspectives to technical discussions.")
creative_strategist.set_modality_flag("LLM_SYSTEM_PROMPT_FLAG", True)
creative_strategist.set_modality_flag("AGENT_FLAG", True)

creative_strategist_id = graphiti_framework.create_agent(
    agent_config=creative_strategist.get_config(),
    agent_name="Creative Strategist", 
    description="Innovation-focused strategy expert",
    tags=["creative", "strategy", "innovation"]
)

print(f"✅ Created agents:")
print(f"   • Research Assistant: {agent_id}")
print(f"   • Technical Expert: {tech_expert_id}")
print(f"   • Creative Strategist: {creative_strategist_id}")

# Generate multi-agent conversation
print(f"\n🎭 Generating multi-agent conversation...")

conversation = conversation_generator.generate_conversation(
    agents=[agent_id, tech_expert_id, creative_strategist_id],
    topic="Building Privacy-Preserving AI Systems with Local Models",
    num_turns=9,  # 3 turns per agent
    context={
        "focus": "ollama_graphiti_integration",
        "goal": "practical_implementation_guide",
        "audience": "developers_and_researchers"
    }
)

print(f"✅ Generated conversation: {conversation['conversation_id']}")
print(f"📝 Topic: {conversation['topic']}")
print(f"👥 Participants: {len(conversation['participants'])} agents")
print(f"🔄 Turns: {len(conversation['turns'])} exchanges")

# Show conversation preview
print(f"\n💬 Conversation Preview:")
for i, turn in enumerate(conversation['turns'][:6], 1):  # Show first 6 turns
    agent_name = "Research Assistant" if turn['agent_id'] == agent_id else \
                 "Technical Expert" if turn['agent_id'] == tech_expert_id else \
                 "Creative Strategist"
    
    content_preview = turn['content'][:80] + "..." if len(turn['content']) > 80 else turn['content']
    print(f"   {i}. 🤖 {agent_name}: {content_preview}")

# Export conversation to JSONL for training
export_path = "multi_agent_ollama_conversation.jsonl"
conversation_generator.export_conversation_jsonl(
    conversation_id=conversation['conversation_id'],
    output_path=export_path,
    include_metadata=True
)

print(f"\n📊 Exported to: {export_path}")
print(f"🎯 Use case: Training data for multi-agent AI systems")
print(f"📈 Format: JSONL with agent personalities and Ollama integration context")

## 📊 Part 6: System Analytics and Data Export

Let's analyze what we've built and explore the data export capabilities:
- **Agent Statistics** - Review created agents and their configurations
- **Knowledge Base Analysis** - Examine stored knowledge and embeddings
- **Conversation Metrics** - Analyze multi-agent interaction patterns  
- **Export Options** - Demonstrate various data export formats
- **Performance Insights** - Show Polars database efficiency

In [ ]:
# Comprehensive system analytics and data export demonstration
print("📈 AMS-DB System Analytics Dashboard")
print("=" * 50)

# 1. Agent Analytics
print("\n🤖 Agent Statistics:")
agents_df = db_handler.agent_matrix
print(f"   • Total agents created: {agents_df.height}")

for row in agents_df.iter_rows(named=True):
    agent_name = row["agent_name"] 
    created_at = row["created_at"]
    config_size = len(row["config_json"])
    print(f"   • {agent_name}: Created {created_at}, Config: {config_size:,} chars")

# 2. Knowledge Base Analytics
print(f"\n📚 Knowledge Base Statistics:")
knowledge_df = db_handler.knowledge_base
print(f"   • Total knowledge documents: {knowledge_df.height}")

if knowledge_df.height > 0:
    # Group by agent
    knowledge_by_agent = knowledge_df.group_by("agent_id").agg([
        knowledge_df.select("content").count().alias("doc_count"),
        knowledge_df.select("content").str.len_chars().mean().alias("avg_length")
    ])
    
    for row in knowledge_by_agent.iter_rows(named=True):
        agent_id_short = row["agent_id"][:20] + "..." if len(row["agent_id"]) > 20 else row["agent_id"]
        print(f"   • {agent_id_short}: {row['doc_count']} docs, {row['avg_length']:.0f} chars avg")

# 3. Conversation Analytics
print(f"\n💬 Conversation Statistics:")
conversations_df = db_handler.conversation_history
print(f"   • Total messages: {conversations_df.height}")

if conversations_df.height > 0:
    # Messages by agent
    messages_by_agent = conversations_df.group_by("agent_id").agg([
        conversations_df.select("content").count().alias("message_count")
    ]).sort("message_count", descending=True)
    
    for row in messages_by_agent.iter_rows(named=True):
        agent_id_short = row["agent_id"][:20] + "..." if len(row["agent_id"]) > 20 else row["agent_id"]
        print(f"   • {agent_id_short}: {row['message_count']} messages")
    
    # Session analytics
    unique_sessions = conversations_df.select("session_id").n_unique()
    print(f"   • Unique sessions: {unique_sessions}")

# 4. Database Performance Metrics
print(f"\n⚡ Database Performance:")
db_stats = db_handler.get_database_stats()
print(f"   • Database size: {db_stats['database_size_mb']:.2f} MB")
print(f"   • Query speed: Polars high-performance backend")
print(f"   • Storage format: Parquet (columnar, compressed)")

# 5. Export Demonstrations
print(f"\n📤 Data Export Demonstrations:")

# Export agents as JSON for sharing configurations
agents_export_path = "ams_agents_export.json"
agents_data = []
for row in agents_df.iter_rows(named=True):
    agents_data.append({
        "agent_id": row["agent_id"],
        "agent_name": row["agent_name"], 
        "description": row["description"],
        "created_at": str(row["created_at"]),
        "config": json.loads(row["config_json"])
    })

with open(agents_export_path, 'w') as f:
    json.dump(agents_data, f, indent=2, default=str)
print(f"   ✅ Agents exported: {agents_export_path}")

# Export knowledge base as CSV for analysis
if knowledge_df.height > 0:
    knowledge_export_path = "ams_knowledge_export.csv"
    knowledge_df.write_csv(knowledge_export_path)
    print(f"   ✅ Knowledge base exported: {knowledge_export_path}")

# Export conversations as Parquet for high-performance storage
if conversations_df.height > 0:
    conversations_export_path = "ams_conversations_export.parquet"
    conversations_df.write_parquet(conversations_export_path)
    print(f"   ✅ Conversations exported: {conversations_export_path}")

# 6. Integration Summary
print(f"\n🎯 Integration Summary:")
print(f"   ✅ Ollama Models: phi4:latest, gemma3:4b, nomic-embed-text")
print(f"   ✅ Graphiti Integration: Knowledge graphs with temporal memory")
print(f"   ✅ Polars Database: High-speed data operations")
print(f"   ✅ Agent System: Complete configuration management")
print(f"   ✅ Privacy: All processing local (no external API calls)")
print(f"   ✅ Export Formats: JSON, CSV, Parquet, JSONL")

# 7. Next Steps for Users
print(f"\n🚀 Next Steps:")
print(f"   1. Experiment with different Ollama models")
print(f"   2. Create domain-specific agent configurations")
print(f"   3. Build larger knowledge bases for your use case")
print(f"   4. Generate training datasets for model fine-tuning")
print(f"   5. Integrate with your existing AI workflows")

print(f"\n🎉 AMS-DB Ollama + Graphiti demonstration complete!")
print(f"📖 This notebook showed the complete integration of:")
print(f"   • Local LLM processing with Ollama")
print(f"   • Temporal knowledge graphs with Graphiti") 
print(f"   • High-speed database operations with Polars")
print(f"   • Comprehensive agent configuration management")
print(f"   • Multi-agent conversation generation")
print(f"   • Data export for ML workflows")

# Build a ShoeBot Sales Agent using LangGraph and Graphiti

The following example demonstrates building an agent using LangGraph. Graphiti is used to personalize agent responses based on information learned from prior conversations. Additionally, a database of products is loaded into the Graphiti graph, enabling the agent to speak to these products.

The agent implements:
- persistence of new chat turns to Graphiti and recall of relevant Facts using the most recent message.
- a tool for querying Graphiti for shoe information
- an in-memory MemorySaver to maintain agent state.

## Install dependencies
```shell
pip install graphiti-core langchain-openai langgraph ipywidgets
```

Ensure that you've followed the Graphiti installation instructions. In particular, installation of `neo4j`.

In [ ]:
import asyncio
import json
import logging
import os
import sys
import uuid
from contextlib import suppress
from datetime import datetime, timezone
from pathlib import Path
from typing import Annotated

import ipywidgets as widgets
from dotenv import load_dotenv
from IPython.display import Image, display
from typing_extensions import TypedDict

load_dotenv()

In [ ]:
def setup_logging():
    logger = logging.getLogger()
    logger.setLevel(logging.ERROR)
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    return logger


logger = setup_logging()

## LangSmith integration (Optional)

If you'd like to trace your agent using LangSmith, ensure that you have a `LANGSMITH_API_KEY` set in your environment.

Then set `os.environ['LANGCHAIN_TRACING_V2'] = 'false'` to `true`.


In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'false'
os.environ['LANGCHAIN_PROJECT'] = 'Graphiti LangGraph Tutorial'

## Configure Graphiti

Ensure that you have `neo4j` running and a database created. Ensure that you've configured the following in your environment.

```bash
NEO4J_URI=
NEO4J_USER=
NEO4J_PASSWORD=
```

In [ ]:
# Configure Graphiti

from graphiti_core import Graphiti
from graphiti_core.edges import EntityEdge
from graphiti_core.nodes import EpisodeType
from graphiti_core.utils.maintenance.graph_data_operations import clear_data

neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', 'password')

client = Graphiti(
    neo4j_uri,
    neo4j_user,
    neo4j_password,
)

## Generating a database schema 

The following is only required for the first run of this notebook or when you'd like to start your database over.

**IMPORTANT**: `clear_data` is destructive and will wipe your entire database.

In [ ]:
# Note: This will clear the database
await clear_data(client.driver)
await client.build_indices_and_constraints()

## Load Shoe Data into the Graph

Load several shoe and related products into the Graphiti. This may take a while.


**IMPORTANT**: This only needs to be done once. If you run `clear_data` you'll need to rerun this step.

In [ ]:
async def ingest_products_data(client: Graphiti):
    script_dir = Path.cwd().parent
    json_file_path = script_dir / 'data' / 'manybirds_products.json'

    with open(json_file_path) as file:
        products = json.load(file)['products']

    for i, product in enumerate(products):
        await client.add_episode(
            name=product.get('title', f'Product {i}'),
            episode_body=str({k: v for k, v in product.items() if k != 'images'}),
            source_description='ManyBirds products',
            source=EpisodeType.json,
            reference_time=datetime.now(timezone.utc),
        )


await ingest_products_data(client)

## Create a user node in the Graphiti graph

In your own app, this step could be done later once the user has identified themselves and made their sales intent known. We do this here so we can configure the agent with the user's `node_uuid`.

In [ ]:
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_EPISODE_MENTIONS

user_name = 'jess'

await client.add_episode(
    name='User Creation',
    episode_body=(f'{user_name} is interested in buying a pair of shoes'),
    source=EpisodeType.text,
    reference_time=datetime.now(timezone.utc),
    source_description='SalesBot',
)

# let's get Jess's node uuid
nl = await client._search(user_name, NODE_HYBRID_SEARCH_EPISODE_MENTIONS)

user_node_uuid = nl.nodes[0].uuid

# and the ManyBirds node uuid
nl = await client._search('ManyBirds', NODE_HYBRID_SEARCH_EPISODE_MENTIONS)
manybirds_node_uuid = nl.nodes[0].uuid

In [ ]:
def edges_to_facts_string(entities: list[EntityEdge]):
    return '-' + '\n- '.join([edge.fact for edge in entities])

In [ ]:
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, add_messages
from langgraph.prebuilt import ToolNode

## `get_shoe_data` Tool

The agent will use this to search the Graphiti graph for information about shoes. We center the search on the `manybirds_node_uuid` to ensure we rank shoe-related data over user data.


In [ ]:
@tool
async def get_shoe_data(query: str) -> str:
    """Search the graphiti graph for information about shoes"""
    edge_results = await client.search(
        query,
        center_node_uuid=manybirds_node_uuid,
        num_results=10,
    )
    return edges_to_facts_string(edge_results)


tools = [get_shoe_data]
tool_node = ToolNode(tools)

In [ ]:
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0).bind_tools(tools)

In [ ]:
# Test the tool node
await tool_node.ainvoke({'messages': [await llm.ainvoke('wool shoes')]})

## Chatbot Function Explanation

The chatbot uses Graphiti to provide context-aware responses in a shoe sales scenario. Here's how it works:

1. **Context Retrieval**: It searches the Graphiti graph for relevant information based on the latest message, using the user's node as the center point. This ensures that user-related facts are ranked higher than other information in the graph.

2. **System Message**: It constructs a system message incorporating facts from Graphiti, setting the context for the AI's response.

3. **Knowledge Persistence**: After generating a response, it asynchronously adds the interaction to the Graphiti graph, allowing future queries to reference this conversation.

This approach enables the chatbot to maintain context across interactions and provide personalized responses based on the user's history and preferences stored in the Graphiti graph.

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    user_name: str
    user_node_uuid: str


async def chatbot(state: State):
    facts_string = None
    if len(state['messages']) > 0:
        last_message = state['messages'][-1]
        graphiti_query = f'{"SalesBot" if isinstance(last_message, AIMessage) else state["user_name"]}: {last_message.content}'
        # search graphiti using Jess's node uuid as the center node
        # graph edges (facts) further from the Jess node will be ranked lower
        edge_results = await client.search(
            graphiti_query, center_node_uuid=state['user_node_uuid'], num_results=5
        )
        facts_string = edges_to_facts_string(edge_results)

    system_message = SystemMessage(
        content=f"""You are a skillfull shoe salesperson working for ManyBirds. Review information about the user and their prior conversation below and respond accordingly.
        Keep responses short and concise. And remember, always be selling (and helpful!)

        Things you'll need to know about the user in order to close a sale:
        - the user's shoe size
        - any other shoe needs? maybe for wide feet?
        - the user's preferred colors and styles
        - their budget

        Ensure that you ask the user for the above if you don't already know.

        Facts about the user and their conversation:
        {facts_string or 'No facts about the user and their conversation'}"""
    )

    messages = [system_message] + state['messages']

    response = await llm.ainvoke(messages)

    # add the response to the graphiti graph.
    # this will allow us to use the graphiti search later in the conversation
    # we're doing async here to avoid blocking the graph execution
    asyncio.create_task(
        client.add_episode(
            name='Chatbot Response',
            episode_body=f'{state["user_name"]}: {state["messages"][-1]}\nSalesBot: {response.content}',
            source=EpisodeType.message,
            reference_time=datetime.now(timezone.utc),
            source_description='Chatbot',
        )
    )

    return {'messages': [response]}

## Setting up the Agent

This section sets up the Agent's LangGraph graph:

1. **Graph Structure**: It defines a graph with nodes for the agent (chatbot) and tools, connected in a loop.

2. **Conditional Logic**: The `should_continue` function determines whether to end the graph execution or continue to the tools node based on the presence of tool calls.

3. **Memory Management**: It uses a MemorySaver to maintain conversation state across turns. This is in addition to using Graphiti for facts.

In [ ]:
graph_builder = StateGraph(State)

memory = MemorySaver()


# Define the function that determines whether to continue or not
async def should_continue(state, config):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return 'end'
    # Otherwise if there is, we continue
    else:
        return 'continue'


graph_builder.add_node('agent', chatbot)
graph_builder.add_node('tools', tool_node)

graph_builder.add_edge(START, 'agent')
graph_builder.add_conditional_edges('agent', should_continue, {'continue': 'tools', 'end': END})
graph_builder.add_edge('tools', 'agent')

graph = graph_builder.compile(checkpointer=memory)

Our LangGraph agent graph is illustrated below.

In [ ]:
with suppress(Exception):
    display(Image(graph.get_graph().draw_mermaid_png()))

## Running the Agent

Let's test the agent with a single call

In [ ]:
await graph.ainvoke(
    {
        'messages': [
            {
                'role': 'user',
                'content': 'What sizes do the TinyBirds Wool Runners in Natural Black come in?',
            }
        ],
        'user_name': user_name,
        'user_node_uuid': user_node_uuid,
    },
    config={'configurable': {'thread_id': uuid.uuid4().hex}},
)

## Viewing the Graph

At this stage, the graph would look something like this. The `jess` node is `INTERESTED_IN` the `TinyBirds Wool Runner` node. The image below was generated using Neo4j Desktop.

In [ ]:
display(Image(filename='tinybirds-jess.png', width=850))

## Running the Agent interactively

The following code will run the agent in an event loop. Just enter a message into the box and click submit.

In [ ]:
conversation_output = widgets.Output()
config = {'configurable': {'thread_id': uuid.uuid4().hex}}
user_state = {'user_name': user_name, 'user_node_uuid': user_node_uuid}


async def process_input(user_state: State, user_input: str):
    conversation_output.append_stdout(f'\nUser: {user_input}\n')
    conversation_output.append_stdout('\nAssistant: ')

    graph_state = {
        'messages': [{'role': 'user', 'content': user_input}],
        'user_name': user_state['user_name'],
        'user_node_uuid': user_state['user_node_uuid'],
    }

    try:
        async for event in graph.astream(
            graph_state,
            config=config,
        ):
            for value in event.values():
                if 'messages' in value:
                    last_message = value['messages'][-1]
                    if isinstance(last_message, AIMessage) and isinstance(
                        last_message.content, str
                    ):
                        conversation_output.append_stdout(last_message.content)
    except Exception as e:
        conversation_output.append_stdout(f'Error: {e}')


def on_submit(b):
    user_input = input_box.value
    input_box.value = ''
    asyncio.create_task(process_input(user_state, user_input))


input_box = widgets.Text(placeholder='Type your message here...')
submit_button = widgets.Button(description='Send')
submit_button.on_click(on_submit)

conversation_output.append_stdout('Asssistant: Hello, how can I help you find shoes today?')

display(widgets.VBox([input_box, submit_button, conversation_output]))